In [ ]:
from models import Detail

from psycopg2.extensions import cursor as CursorType


class SearchPartCRMTool:
    
    def search_at_crm_db(self, input: str):
        

class DetailInfoRepository:

    def __init__(self, cursor, vector_collection_name, similarity_search_limit):
        self.cursor: CursorType = cursor
        self.vector_collection_name = vector_collection_name
        self.similarity_search_limit = similarity_search_limit

    def get_top_relevant_messages(self, embeddings, k=3):
        try:
            query = f"""
                WITH vector_matches AS (
                    SELECT brand_id, name, embedding <=> '{embeddings}' AS distance
                    FROM {self.vector_collection_name}
                )
                SELECT brand_id, name, distance
                FROM vector_matches
                ORDER BY distance
                LIMIT '{k}';
            """
            
            self.cursor.execute(query)
            all_matches = self.cursor.fetchall()
            
            relevant_matches = []
            print('All matches:')
            for doc in all_matches:
                print(f'-- {round(doc[2], 2)}: {doc[1]}')
                
                if round(doc[2], 2) <= float(self.similarity_search_limit):
                    relevant_matches.append({
                        "document": doc,
                        "score": doc[2]
                        })

            if len(relevant_matches) == 0:
                print("No relevant matches found")
            else:
                print("Relevant matches: ")
                [print(f'-- {round(doc["score"], 2)}: {doc["document"][2]}') for doc in relevant_matches]
            return relevant_matches
        except Exception as e:
            print(f"[get_top_relevant_messages] {type(e).__name__} exception: {e}")
            return []

    def select_detail_by_ids(self, details_ids):
        details_ids_str = ', '.join([str(d) for d in details_ids])
        query = f"select * from details_info where id IN({details_ids_str})"

        self.cursor.execute(query)

        all_matches = self.cursor.fetchall()

        details = []
        for match in all_matches:
            details.append(Detail(
                id=match[0],
                part_number=match[1],
                brand_name=match[4],
                description=match[5]
            ))
        return details

    def select_detail_by_part_number(self, part_number: str):
        query = f"select * from details_info where part_number='{part_number}'"

        try:
            self.cursor.execute(query)

            all_matches = self.cursor.fetchall()

        
            details = []
            for match in all_matches:
                details.append(Detail(
                    id=match[0],
                    part_number=match[1],
                    brand_name=match[4],
                    description=match[5]
                ))
            return details
        except:
            return []
        
    def select_brands_details(self, brand_ids):
        ids_str = ', '.join([str(b) for b in brand_ids])

        query = f"select * from details_info where brand_id IN({ids_str})"

        self.cursor.execute(query)

        return self.cursor.fetchall()

In [6]:
import openai
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

True

In [7]:
client = openai.Client()

In [1]:
PREFIX = """Solve the following tasks as best you can. You have access to the following tools:"""

FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""

SUFFIX = """Begin!

Task: {input}
Thought:{agent_scratchpad}"""

In [2]:
question = """
Here is the customer request part/parts:

Amount: 7
Brand name: ENDRESS+HAUSER
Part number: 71035522
Country: Chile

Please gather more info about price, technical info, manufacturer at company CRM database and Google.

"""

In [3]:
tools_str = """
    Search part at CRM: This tool is needed for search part at company CRM database, in 40% cases requisted detail by customer already exist at DB.
    The input to this tool is a dictionary specifying the fields of the Search task, depends of data in customer request and will be passed into search CRM `search_part_crm` function.
    Only add fields described by user
    
    Here are few task descriptions and corresponding input examples:
    Task: search part with brand 'Endress+Hauser' and part number '71035522'
    Example Input: {{ "brand": "Endress+Hauser", "part_number": "71035522"}}
    Task: search part with brand "INFRANOR" and model "CD1"
    Example Input: {{ "brand": "INFRANOR", "part_number": "CD1"}}
    
    
    Search part at Google: This tool is needed for search part at Google, in 60% cases requisted detail by customer do not exist at CRM DB and needed to be searched at Google.
    The input to this tool is a dictionary specifying the fields of the Search task, depends of data in customer request and will be passed into search CRM `search_part_google` function.
    Only add fields described by user.
    
    Here are few task descriptions and corresponding input examples:
    Task: search part with brand 'Endress+Hauser' and part number '71035522'
    Example Input: {{ "brand": "Endress+Hauser", "part_number": "71035522"}}
    Task: search part with brand "INFRANOR" and model "CD1"
    Example Input: {{ "brand": "INFRANOR", "part_number": "CD1"}}

"""

In [4]:
prompt = '\n\n'.join([
    PREFIX,
    tools_str,
    FORMAT_INSTRUCTIONS.format(tool_names='Search part at CRM, Search part at Google'),
    SUFFIX.format(input=question, agent_scratchpad='')
])

In [9]:
stop = ['\nObservation:', '\n\tObservation:']

In [10]:
model1 = 'gpt-4-1106-preview'
model2 = 'gpt-4'
model = model1

completion = client.chat.completions.create(model=model, 
        messages=[
        {"role": "user", "content": prompt}
    ], 
    stop=stop,
    temperature=0)

output = completion.choices[0].message.content

print(completion.choices[0].message.content )

Thought: The customer is requesting information on a specific part from the brand ENDRESS+HAUSER with the part number 71035522. They need 7 units and are located in Chile. They want to know the price, technical information, and manufacturer details. Since there's a 40% chance that the part exists in the company CRM database, I should start by searching there before moving on to Google.

Action: Search part at CRM
Action Input: { "brand": "ENDRESS+HAUSER", "part_number": "71035522" }


In [12]:
from langchain.agents.mrkl.output_parser import MRKLOutputParser

In [13]:
output_parser = MRKLOutputParser()

In [16]:
agent_action = output_parser.parse(output)
agent_action

AgentAction(tool='Search part at CRM', tool_input='{ "brand": "ENDRESS+HAUSER", "part_number": "71035522" }', log='Thought: The customer is requesting information on a specific part from the brand ENDRESS+HAUSER with the part number 71035522. They need 7 units and are located in Chile. They want to know the price, technical information, and manufacturer details. Since there\'s a 40% chance that the part exists in the company CRM database, I should start by searching there before moving on to Google.\n\nAction: Search part at CRM\nAction Input: { "brand": "ENDRESS+HAUSER", "part_number": "71035522" }')

In [29]:
from pydantic import BaseModel
import json

In [26]:
def parse_tool_input(tool_input: str, args_schema: BaseModel):
    input_args = args_schema
    if isinstance(tool_input, str):
        if input_args is not None:
            key_ = next(iter(input_args.__fields__.keys()))
            input_args.validate({key_: tool_input})
        return tool_input
    else:
        if input_args is not None:
            result = input_args.parse_obj(tool_input)
            return {k: v for k, v in result.dict().items() if k in tool_input}
    return tool_input


In [24]:
class DetailRequest(BaseModel):
    brand: str
    part_number: str


In [21]:
s = DetailRequest

s.__fields__.keys()

/tmp/ipykernel_2639533/1626135491.py:3: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s.__fields__.keys()


dict_keys(['brand_name', 'part_number'])

In [30]:
json.loads(agent_action.tool_input)

{'brand': 'ENDRESS+HAUSER', 'part_number': '71035522'}

In [31]:
parse_tool_input(json.loads(agent_action.tool_input), DetailRequest)

/tmp/ipykernel_2639533/41373246.py:10: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  result = input_args.parse_obj(tool_input)
/tmp/ipykernel_2639533/41373246.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  return {k: v for k, v in result.dict().items() if k in tool_input}


{'brand': 'ENDRESS+HAUSER', 'part_number': '71035522'}

In [33]:
DetailRequest.parse_obj(json.loads(agent_action.tool_input))

/tmp/ipykernel_2639533/4242616663.py:1: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  DetailRequest.parse_obj(json.loads(agent_action.tool_input))


DetailRequest(brand='ENDRESS+HAUSER', part_number='71035522')

In [ ]:
search_part_at_crm()

In [ ]:

if agent_action == 'Search part at CRM':
    parsed_tool_input = parse_tool_input(json.loads(agent_action.tool_input), DetailRequest)
    tool_input_model = DetailRequest.parse_obj(parsed_tool_input)
    search_part_at_crm(tool_input_model)
    
elif agent_action == 'Search part at Google':
    parsed_tool_input = parse_tool_input(json.loads(agent_action.tool_input), DetailRequest)
    tool_input_model = DetailRequest.parse_obj(parsed_tool_input)
    search_at_google(tool_input_model)

